In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score, permutation_test_score
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.combine import SMOTETomek
from scipy.stats import uniform, randint
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.utils import resample
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [5]:
df = pd.read_csv('C:/Users/staya/OneDrive/Documents/Python Notebooks/GL PGP DSE/Capstone/CSV Files/Diabetes_Preprocessed_Before_Feature_Selection.csv')
df.head()

,race,gender,age,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,diabetesMed,readmitted,preceding_year_visits,number_changes,insulin_treatment
0,Caucasian,0,5,Not Available,Referral,1,41,0,1,Diabetes,Not Required,Not Required,1,None,None,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0,no_med
1,Caucasian,0,15,Discharged to home,Emergency,3,59,0,18,"Endocrine, Nutritional, Metabolic, Immunity",Diabetes,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,-2,-2,-2,-2,1,1,0,0,1,insulin_only
2,AfricanAmerican,0,25,Discharged to home,Emergency,2,11,5,13,"Pregnancy, Childbirth",Diabetes,External causes of injury,6,None,None,-2,-2,-2,0,-2,-2,-2,-2,1,0,3,0,other_meds
3,Caucasian,1,35,Discharged to home,Emergency,2,44,1,16,Infectious and Parasitic,Diabetes,Circulatory,7,None,None,-2,-2,-2,-2,-2,-2,-2,1,1,0,0,1,insulin_only
4,Caucasian,1,45,Discharged to home,Emergency,1,51,0,8,Neoplasms,Neoplasms,Diabetes,5,None,None,-2,-2,-2,0,-2,-2,-2,0,1,0,0,0,insulin_combo


In [6]:
df.shape

(97070, 28)

In [7]:
X = df.drop(['readmitted'], 1)
y = df['readmitted']
X_dum = pd.get_dummies(X, drop_first = True)
X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size = 0.3, random_state = 0, stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((67949, 95), (29121, 95), (67949,), (29121,))

In [8]:
X_train.head()

,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,metformin,repaglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,diabetesMed,preceding_year_visits,number_changes,race_Asian,race_Caucasian,race_Hispanic,race_Other,discharge_disposition_id_Discharged to home with home health service,discharge_disposition_id_Left AMA,discharge_disposition_id_Not Available,discharge_disposition_id_Still patient/referred to this institution,discharge_disposition_id_Transferred to another medical facility,admission_source_id_Not Available,admission_source_id_Referral,admission_source_id_Transferred from another health care facility,diag_1_Circulatory,diag_1_Congenital Anomalies,diag_1_Diabetes,diag_1_Digestive,"diag_1_Endocrine, Nutritional, Metabolic, Immunity",diag_1_External causes of injury,diag_1_Genitourinary,diag_1_Infectious and Parasitic,diag_1_Injury and Poisoning,diag_1_Mental Disorders,diag_1_Musculoskeletal System and Connective Tissue,diag_1_Neoplasms,diag_1_Nervous,diag_1_Other Symptoms,"diag_1_Pregnancy, Childbirth",diag_1_Respiratory,diag_1_Sense Organs,diag_1_Skin and Subcutaneous Tissue,diag_2_Circulatory,diag_2_Congenital Anomalies,diag_2_Diabetes,diag_2_Digestive,"diag_2_Endocrine, Nutritional, Metabolic, Immunity",diag_2_External causes of injury,diag_2_Genitourinary,diag_2_Infectious and Parasitic,diag_2_Injury and Poisoning,diag_2_Mental Disorders,diag_2_Musculoskeletal System and Connective Tissue,diag_2_Neoplasms,diag_2_Nervous,diag_2_Not Required,diag_2_Other Symptoms,"diag_2_Pregnancy, Childbirth",diag_2_Respiratory,diag_2_Sense Organs,diag_2_Skin and Subcutaneous Tissue,diag_3_Circulatory,diag_3_Congenital Anomalies,diag_3_Diabetes,diag_3_Digestive,"diag_3_Endocrine, Nutritional, Metabolic, Immunity",diag_3_External causes of injury,diag_3_Genitourinary,diag_3_Infectious and Parasitic,diag_3_Injury and Poisoning,diag_3_Mental Disorders,diag_3_Musculoskeletal System and Connective Tissue,diag_3_Neoplasms,diag_3_Nervous,diag_3_Not Required,diag_3_Other Symptoms,"diag_3_Pregnancy, Childbirth",diag_3_Respiratory,diag_3_Sense Organs,diag_3_Skin and Subcutaneous Tissue,max_glu_serum_>300,max_glu_serum_None,max_glu_serum_Norm,A1Cresult_>8,A1Cresult_None,A1Cresult_Norm,insulin_treatment_insulin_only,insulin_treatment_no_med,insulin_treatment_other_meds
73921,0,45,6,53,1,29,8,-1,-2,0,-2,-2,-2,-2,-1,1,2,2,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0
96919,0,85,2,43,0,24,7,-2,-2,-2,0,-2,-2,-2,0,1,3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
25853,1,65,1,1,1,17,3,-2,-2,-2,-2,-2,-2,-2,-2,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
31598,1,85,5,42,0,20,9,-2,-2,-2,1,-2,-2,-2,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
24086,1,65,3,45,2,17,8,-2,-2,-2,-2,-2,-2,-2,-2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


In [9]:
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

## Building Model and Making Predictions

model = LGBMClassifier(objective = 'binary', random_state = 0, n_jobs = -1, is_unbalance = True,
                       colsample_bytree= 0.7, learning_rate= 0.055499999999999994, min_child_samples= 28,
                       n_estimators= 180, reg_alpha= 50, reg_lambda= 5, subsample= 0.7, subsample_freq= 2)

X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
print("Training Time:", end_time - start_time)
y_train_pred = model.predict(X_train)
y_train_prob = model.predict_proba(X_train)[:, 1]
start_time = time.time()
y_test_pred = model.predict(X_test)
end_time = time.time()
print("Prediction Time:", end_time - start_time)
y_test_prob = model.predict_proba(X_test)[:, 1]
print("Train Accuracy Score:", accuracy_score(y_train, y_train_pred))
print("Train Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred), '\n')
print("Train F1 Score:", f1_score(y_train, y_train_pred))
print("Train Precision Score:", precision_score(y_train, y_train_pred))
print("Train Recall Score:", recall_score(y_train, y_train_pred))
print("Train ROC_AUC Score:", roc_auc_score(y_train, y_train_prob))
print()
print("Test Accuracy Score:", accuracy_score(y_test, y_test_pred))
print("Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred), '\n')
print("Test F1 Score:", f1_score(y_test, y_test_pred))
print("Test Precision Score:", precision_score(y_test, y_test_pred))
print("Test Recall Score:", recall_score(y_test, y_test_pred))
print("Test ROC_AUC Score:", roc_auc_score(y_test, y_test_prob))

Training Time: 5.15981912612915
Prediction Time: 0.5377452373504639
Train Accuracy Score: 0.6216279856951538
Train Confusion Matrix:
 [[37093 23072]
 [ 2638  5146]] 

Train F1 Score: 0.2858730070551636
Train Precision Score: 0.18236586575944433
Train Recall Score: 0.6610996916752312
Train ROC_AUC Score: 0.6966109888027179

Test Accuracy Score: 0.6164280072799698
Test Confusion Matrix:
 [[15867  9918]
 [ 1252  2084]] 

Test F1 Score: 0.2717433824488199
Test Precision Score: 0.1736377270454924
Test Recall Score: 0.6247002398081535
Test ROC_AUC Score: 0.6662794894974073


In [10]:
sorted(zip(model.feature_importances_, X_train.columns), reverse = True)

[(661, 'num_medications'),
 (619, 'num_lab_procedures'),
 (466, 'time_in_hospital'),
 (353, 'age'),
 (285, 'preceding_year_visits'),
 (241, 'number_diagnoses'),
 (223, 'num_procedures'),
 (87, 'insulin'),
 (85, 'discharge_disposition_id_Transferred_to_another_medical_facility'),
 (81, 'metformin'),
 (67, 'diag_1_Respiratory'),
 (64, 'diag_3_External_causes_of_injury'),
 (55, 'admission_source_id_Not_Available'),
 (52, 'glimepiride'),
 (50, 'diag_3_Endocrine__Nutritional__Metabolic__Immunity'),
 (50, 'diag_1_Musculoskeletal_System_and_Connective_Tissue'),
 (50, 'A1Cresult_None'),
 (49, 'diabetesMed'),
 (49, 'admission_source_id_Transferred_from_another_health_care_facility'),
 (47, 'rosiglitazone'),
 (46, 'number_changes'),
 (46, 'diag_1_Genitourinary'),
 (45, 'discharge_disposition_id_Not_Available'),
 (44, 'discharge_disposition_id_Discharged_to_home_with_home_health_service'),
 (44, 'diag_2_Neoplasms'),
 (44, 'diag_1_Other_Symptoms'),
 (43, 'diag_1_Circulatory'),
 (42, 'pioglitazone'

In [11]:
fi = pd.DataFrame({'FI':model.feature_importances_, 'Feature':X_train.columns})
features = fi[fi['FI'] >= 80]['Feature'].values
features, len(features)

(array(['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
        'num_medications', 'number_diagnoses', 'metformin', 'insulin',
        'preceding_year_visits',
        'discharge_disposition_id_Transferred_to_another_medical_facility'],
       dtype=object),
 10)

In [12]:
X_train1 = X_train[features]
X_test1 = X_test[features]
X_train1.shape, X_test1.shape

((67949, 10), (29121, 10))

In [14]:
model1 = LGBMClassifier(objective = 'binary', random_state = 0, n_jobs = -1, is_unbalance = True,
                       colsample_bytree= 0.7, learning_rate= 0.055499999999999994, min_child_samples= 28,
                       n_estimators= 180, reg_alpha= 50, reg_lambda= 5, subsample= 0.7, subsample_freq= 2)

start_time = time.time()
model1.fit(X_train1, y_train)
end_time = time.time()
print("Training Time:", end_time - start_time)
y_train_pred = model1.predict(X_train1)
y_train_prob = model1.predict_proba(X_train1)[:, 1]
start_time = time.time()
y_test_pred = model1.predict(X_test1)
end_time = time.time()
print("Prediction Time:", end_time - start_time)
y_test_prob = model1.predict_proba(X_test1)[:, 1]
print("Train Accuracy Score:", accuracy_score(y_train, y_train_pred))
print("Train Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred), '\n')
print("Train F1 Score:", f1_score(y_train, y_train_pred))
print("Train Precision Score:", precision_score(y_train, y_train_pred))
print("Train Recall Score:", recall_score(y_train, y_train_pred))
print("Train ROC_AUC Score:", roc_auc_score(y_train, y_train_prob))
print()
print("Test Accuracy Score:", accuracy_score(y_test, y_test_pred))
print("Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred), '\n')
print("Test F1 Score:", f1_score(y_test, y_test_pred))
print("Test Precision Score:", precision_score(y_test, y_test_pred))
print("Test Recall Score:", recall_score(y_test, y_test_pred))
print("Test ROC_AUC Score:", roc_auc_score(y_test, y_test_prob))

Training Time: 2.2619197368621826
Prediction Time: 0.41037988662719727
Train Accuracy Score: 0.6117529323463186
Train Confusion Matrix:
 [[36501 23664]
 [ 2717  5067]] 

Train F1 Score: 0.2775297822812543
Train Precision Score: 0.17636002923671296
Train Recall Score: 0.650950668036999
Train ROC_AUC Score: 0.6801195340340612

Test Accuracy Score: 0.608358229456406
Test Confusion Matrix:
 [[15650 10135]
 [ 1270  2066]] 

Test F1 Score: 0.2659458067838064
Test Precision Score: 0.16933038275551185
Test Recall Score: 0.6193045563549161
Test ROC_AUC Score: 0.6551203656039682


In [15]:
import pickle
pickle.dump(model1, open('model.pkl', 'wb'))